In [6]:
# *torch
from pickletools import optimize
# from sched import scheduler
import torch
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler as scheduler
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader

# *transformers
from transformers import MBartForConditionalGeneration, MBartTokenizer, MBartConfig


import utils as utils


# *basic
import os
import time
import shutil
import json, datetime
import numpy as np
from collections import OrderedDict
from tqdm import tqdm
import yaml
import random
import wandb
import copy
from pathlib import Path
import math
import sys
from typing import Iterable, Optional
from loguru import logger

from sacrebleu.metrics import BLEU, CHRF, TER

# *timm
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from timm.utils import NativeScaler
from timm.loss import SoftTargetCrossEntropy
from timm.optim import AdamW

# visualization
from torchvision.utils import save_image, make_grid
from PIL import Image
import argparse
from hpman.m import _
import hpargparse

# global definition
from definition import *

import gzip
import pickle
import torch


In [44]:

from prep_args import * 
parser = argparse.ArgumentParser('Visual-Language-Pretraining (VLP) V2 scripts', parents=[get_args_parser()])

hpargparse.bind(parser, _)
args = parser.parse_args()

ImportError: cannot import name 'get_args_parser' from 'prep_args' (e:\SLT_FYP\data pre-analysis\prep_args.py)

In [10]:
with open(args.config, 'r+', encoding='utf-8') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

config

{'name': 'GFSLT-VLP CSL-Daily',
 'data': {'train': '../../CSL-Daily/sentence_label/processed/labels_train.pkl',
  'dev': '../../CSL-Daily/sentence_label/processed/labels_dev.pkl',
  'test': '../../CSL-Daily/sentence_label/processed/labels_test.pkl',
  'img_path': '../../CSL-Daily/sentence/frames_512x512',
  'max_length': 300},
 'training': {'wandb': 'disabled', 'scale_embedding': False},
 'model': {'transformer': './pretrain_models/CSL/MBart_trimmed',
  'visual_encoder': './pretrain_models/CSL/mytran',
  'sign_proj': True}}

In [15]:
from transformers import MBart50Tokenizer

# Load the MBART tokenizer
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
# Example: Chinese sentences from the CSL-daily dataset
texts = ["这是一个例子句子。", "这是另一个例子句子。"]

# Tokenize the Chinese text data
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
input_ids = tokenized_texts['input_ids']
print(input_ids)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[250004,      6, 100013, 101676,  27683,   1344,     30,      2,      1],
        [250004,      6,   8513,  83757, 101676,  27683,   1344,     30,      2]])


In [37]:
import importlib
import data_classes
importlib.reload(data_classes)
from data_classes import * 

train_dataset = S2T_Dataset(tokenizer, config, args, 'train', training_refurbish = False)

In [41]:
train_dataset[0][1].shape

['000000.jpg', '000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg', '000011.jpg', '000012.jpg', '000013.jpg', '000014.jpg', '000015.jpg', '000016.jpg', '000017.jpg', '000018.jpg', '000019.jpg', '000020.jpg', '000021.jpg', '000022.jpg', '000023.jpg', '000024.jpg', '000025.jpg', '000026.jpg', '000027.jpg', '000028.jpg', '000029.jpg', '000030.jpg', '000031.jpg', '000032.jpg', '000033.jpg', '000034.jpg', '000035.jpg', '000036.jpg', '000037.jpg', '000038.jpg', '000039.jpg', '000040.jpg', '000041.jpg', '000042.jpg', '000043.jpg', '000044.jpg', '000045.jpg', '000046.jpg', '000047.jpg', '000048.jpg', '000049.jpg', '000050.jpg', '000051.jpg']


torch.Size([52, 3, 224, 224])